# Header placeholder

In [1]:
# Check python version, S/B: 3.10 to ensure the yfinance package functions as expected.
import sys

# Print python version
print("Python version:", sys.version)

# Print the executable path (useful to confirm which environment is used)
print("Python executable:", sys.executable) # 'sys.executable' -> Shows the path to the Python interpreter your notebook
# is using, which helps confirm that the kernel is pointing to the correct environment (e.g., 'py310')

Python version: 3.10.19 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 16:41:31) [MSC v.1929 64 bit (AMD64)]
Python executable: c:\Users\ejoaq\anaconda3\envs\py310\python.exe


In [2]:
# Test that ticker data can be pulled.
import yfinance as yf

aapl = yf.Ticker("AAPL")
data = aapl.history(period="5d")
#data = aapl.history(period="1mo") # use 1 month instead of 5 days.
print(data.head())

                                 Open        High         Low       Close  \
Date                                                                        
2025-11-18 00:00:00-05:00  269.989990  270.709991  265.320007  267.440002   
2025-11-19 00:00:00-05:00  265.529999  272.209991  265.500000  268.559998   
2025-11-20 00:00:00-05:00  270.829987  275.429993  265.920013  266.250000   
2025-11-21 00:00:00-05:00  265.950012  273.329987  265.670013  271.489990   
2025-11-24 00:00:00-05:00  270.899994  276.980011  270.899994  275.920013   

                             Volume  Dividends  Stock Splits  
Date                                                          
2025-11-18 00:00:00-05:00  45677300        0.0           0.0  
2025-11-19 00:00:00-05:00  40424500        0.0           0.0  
2025-11-20 00:00:00-05:00  45823600        0.0           0.0  
2025-11-21 00:00:00-05:00  58784100        0.0           0.0  
2025-11-24 00:00:00-05:00  61882174        0.0           0.0  


Use `yf.download` instead of `Ticker.history`; `yf.download` is more robust

In [8]:
data = yf.download("AAPL", period="5d", interval="1d", auto_adjust=True) # `auto_adjust=True` returns adjusted close prices.
print(data.head())

[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open    Volume
Ticker            AAPL        AAPL        AAPL        AAPL      AAPL
Date                                                                
2025-11-18  267.440002  270.709991  265.320007  269.989990  45677300
2025-11-19  268.559998  272.209991  265.500000  265.529999  40424500
2025-11-20  266.250000  275.429993  265.920013  270.829987  45823600
2025-11-21  271.489990  273.329987  265.670013  265.950012  58784100
2025-11-24  275.920013  276.980011  270.899994  270.899994  61882174


## Check network/firewall

Make sure python can reach Yahoo Finance:

In [9]:
import requests
r = requests.get("https://query1.finance.yahoo.com/v7/finance/download/AAPL")
print(r.status_code)

429


200 → network OK

Non-200 → firewall or network blocking Yahoo

## One-Cell Environment Validation Script

This test verifies:

✅ Your notebook is running under the correct Python 3.10 interpreter

✅ All required libraries import without error

✅ The library versions match the ones installed in your new conda environment

✅ yfinance is functional (downloads AAPL data)

✅ There are no import conflicts

If everything prints green checkmarks (✅), then your new environment is fully set up and ready.

In [13]:
#import sys
import importlib

print("🔍 VALIDATING PYTHON ENVIRONMENT\n")

# Check Python version
print(f"Python version: {sys.version}")
print(f"Executable: {sys.executable}\n")

# List of required packages
packages = ["numpy", "pandas", "matplotlib", "yfinance", "statsmodels"]

for pkg in packages:
    try:
        module = importlib.import_module(pkg)
        ver = getattr(module, "__version__", "unknown")
        print(f"✅ {pkg} imported successfully — version: {ver}")
    except Exception as e:
        print(f"❌ Error importing {pkg}: {e}")

# Quick functional test: small yfinance pull
print("\nTesting yfinance data download…")

import yfinance as yf

try:
    test = yf.Ticker("AAPL").history(period="5d")
    if not test.empty:
        print("✅ yfinance: Successfully downloaded sample price data (AAPL, 5 days)")
    else:
        print("❌ yfinance returned an empty dataset")
except Exception as e:
    print(f"❌ yfinance error: {e}")

print("\n🎉 Environment validation complete!")

🔍 VALIDATING PYTHON ENVIRONMENT

Python version: 3.10.19 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 16:41:31) [MSC v.1929 64 bit (AMD64)]
Executable: c:\Users\ejoaq\anaconda3\envs\py310\python.exe

✅ numpy imported successfully — version: 2.2.6
✅ pandas imported successfully — version: 2.3.3
✅ matplotlib imported successfully — version: 3.10.8
✅ yfinance imported successfully — version: 0.2.66
✅ statsmodels imported successfully — version: 0.14.5

Testing yfinance data download…
✅ yfinance: Successfully downloaded sample price data (AAPL, 5 days)

🎉 Environment validation complete!


In [14]:
# plot_arkk_vs_sixt_alpha_beta.py
# Requires: yfinance, pandas, numpy, matplotlib, statsmodels
# Install if needed: pip install yfinance pandas matplotlib statsmodels

import datetime as dt # Provides classes for working with dates and timestamps
import numpy as np # Numerical operations; numerical computing and arrays
import pandas as pd # Data manipulation and analysis using DataFrames
import yfinance as yf # Download financial market data from Yahoo Finance; provides interface for pulling stock, ETF, index, and crypto price history.
import matplotlib.pyplot as plt # Plotting and visualization.
import statsmodels.api as sm # Statistical modeling and econometrics.

In [15]:
# plot_arkk_vs_sixt_alpha_beta.py
# Requires: yfinance, pandas, numpy, matplotlib, statsmodels
# Install if needed: pip install yfinance pandas matplotlib statsmodels

import datetime as dt
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import statsmodels.api as sm

def fetch_data(tickers, start, end):
    data = yf.download(tickers, start=start, end=end, progress=False, auto_adjust=True)
    # auto_adjust=True gives adjusted prices directly; if multiple tickers result is a DataFrame with columns
    if isinstance(data, pd.DataFrame) and 'Close' in data.columns:
        # single ticker case may produce MultiIndex; handle both single & multi
        try:
            close = data['Close']
        except Exception:
            close = data
    else:
        close = data['Close'] if 'Close' in data else data
    return close

def compute_returns(price):
    # daily log returns are common; we'll use simple returns here
    return price.pct_change().dropna()

def annualize(x, periods_per_year=252):
    return ((1 + x.mean()) ** periods_per_year - 1) if isinstance(x, pd.Series) else None

def regress_alpha_beta(y, x):
    # y and x should be aligned pandas Series of returns
    X = sm.add_constant(x.values)  # adds intercept (alpha)
    model = sm.OLS(y.values, X, missing='drop')
    res = model.fit()
    # res.params[0] is alpha (daily), res.params[1] is beta
    alpha_daily = res.params[0]
    beta = res.params[1]
    # standard errors
    se = res.bse
    alpha_se = se[0]
    beta_se = se[1]
    # annualize alpha (approx): alpha_annual ≈ (1 + alpha_daily) ** 252 - 1
    alpha_annual = (1 + alpha_daily) ** 252 - 1
    # For interpretation often people report alpha in annualized percentage points
    return {
        'alpha_daily': alpha_daily,
        'alpha_se_daily': alpha_se,
        'beta': beta,
        'beta_se': beta_se,
        'alpha_annual': alpha_annual,
        'model_summary': res.summary()
    }

def main():
    # Settings
    ticker_arkk = 'ARKK'       # ARK Innovation ETF
    ticker_sixt = '^SIXT'      # Technology Select Sector Index
    end = dt.date.today()
    start = end - dt.timedelta(days=5*365)  # ~5 years
    rf_daily = 0.0  # set to daily risk-free rate if desired (e.g., 0.03/252 for 3% p.a.)

    # Fetch adjusted-close prices
    print(f"Fetching data from {start} to {end}...")
    prices = fetch_data([ticker_arkk, ticker_sixt], start=start, end=end)

    # Ensure both columns exist
    if ticker_arkk not in prices.columns or ticker_sixt not in prices.columns:
        raise RuntimeError("Couldn't fetch both tickers. Inspect the 'prices' df:\n", prices.head())

    prices = prices[[ticker_arkk, ticker_sixt]].dropna()
    prices.columns = ['ARKK', 'SIXT']

    # Normalize for plotting (start at 100)
    normalized = prices / prices.iloc[0] * 100

    # Plot normalized prices
    plt.figure(figsize=(12,6))
    plt.plot(normalized.index, normalized['ARKK'], label='ARKK (normalized)')
    plt.plot(normalized.index, normalized['SIXT'], label='SIXT (Technology Select Sector) (normalized)')
    plt.title('ARKK vs SIXT — Normalized Prices (Start at 100)')
    plt.ylabel('Normalized price (start=100)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Compute daily returns (simple returns)
    returns = compute_returns(prices)
    returns = returns.dropna()

    # Subtract risk-free (if using excess returns). We'll assume rf_daily = 0 unless you set it.
    returns_excess = returns - rf_daily

    y = returns_excess['ARKK']
    x = returns_excess['SIXT']

    # Regress ARKK on SIXT
    res = regress_alpha_beta(y, x)

    print("\n--- Regression results (ARKK returns ~ alpha + beta * SIXT returns) ---")
    print(f"Daily alpha (intercept): {res['alpha_daily']:.6f}")
    print(f"Daily alpha (approx %): {res['alpha_daily']*100:.4f}%")
    print(f"Annualized alpha (approx): {res['alpha_annual']:.6f} -> {res['alpha_annual']*100:.4f}% p.a.")
    print(f"Beta: {res['beta']:.6f}")
    print(f"Alpha SE (daily): {res['alpha_se_daily']:.6f}")
    print(f"Beta SE: {res['beta_se']:.6f}")

    print("\nFull OLS summary:\n")
    print(res['model_summary'])

    # Scatter plot with fitted regression line
    plt.figure(figsize=(8,6))
    plt.scatter(x, y, alpha=0.4, label='Daily returns (points)')
    # fitted line
    x_vals = np.linspace(x.min(), x.max(), 100)
    y_fit = res['alpha_daily'] + res['beta'] * x_vals
    plt.plot(x_vals, y_fit, label=f'Fit: alpha={res["alpha_daily"]:.6f}, beta={res["beta"]:.3f}', linewidth=2)
    plt.xlabel('SIXT daily return')
    plt.ylabel('ARKK daily return')
    plt.title('ARKK vs SIXT — Daily Returns and Linear Fit')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

if __name__ == '__main__':
    main()


Fetching data from 2020-11-25 to 2025-11-24...



1 Failed download:
['^SIXT']: YFPricesMissingError('possibly delisted; no price data found  (1d 2020-11-25 -> 2025-11-24)')


IndexError: single positional indexer is out-of-bounds

### To do:

1. Section 3: Compute returns -- need to display the covariance matrix, to get a quick view of the relationship between assets. -ERODRIGUEZ, TUES 25NOV2025 15:41
2. Section 4: Optimization that maximizes Sharpe ratio, does not factor in a risk-free rate. For a more accurate and "real world" representation of the Sharpe ratio, include it in the optimizer. -ERODRIGUEZ, TUES 25NOV2025 15:35

In [1]:
# diversified_portfolio_with_beta.py
# Paste into Jupyter or run as script.
# Requirements: yfinance, pandas, numpy, matplotlib, statsmodels, scipy
# Install if needed:
# pip install yfinance pandas numpy matplotlib statsmodels scipy

import datetime as dt
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.optimize import minimize

# ----------------------------
# 1) Define sector ETFs + SPY
# ----------------------------

# Choose one ETF per GICS sector for diversified exposure
sector_etfs = {
    "Communication Services": "ESPO", # modified from XLC, ESPO - VanEck Video Gaming and eSports ETF for more focused exposure
    "Consumer Discretionary": "ONLN", # modified from XLY, ONLN - ProShares Online Retail ETF -- AMZN is a major holding
    "Consumer Staples": "KXI", # modified from XLP, KXI - Consumer Staples Select Sector SPDR Fund
    "Energy": "URA", # modified from XLE, URA - Uranium ETF for energy exposure; Uranium is a key component of nuclear energy, which is a major energy source for the rising data center industry.
    "Financials": "ARKF", # modified from XLF, ARKF - ARK Fintech Innovation ETF for more growth-oriented financial exposure; also, ARKF has a higher beta than XLF, which may be desirable for growth. Financial tech is cool, blockchain and all that.
    "Health Care": "ARKG", # modified from XLV, ARKG - ARK Genomic Revolution ETF for innovative healthcare exposure
    "Industrials": "ARKX", # modified from XLI, ARKX - ARK Space Exploration & Innovation ETF for industrial exposure focused on aerospace and innovation.
    "Information Technology": "QTUM", # modified from XLK, QTUM - Quantum Computing ETF for cutting-edge tech exposure; quantum computing is the future.
    "Materials": "GOAU", # modified from XLB, GOAU - Gold and Precious Metals ETF for materials exposure; gold is a traditional hedge and important material.
    "Real Estate": "RWO", # modified from XLRE, RWO - SPDR Dow Jones Global Real Estate ETF for global real estate exposure.
    "Utilities": "UTES", # modified from XLU, UTES - Global X U.S. Infrastructure Development ETF for utilities and infrastructure exposure.
}
sectors = list(sector_etfs.keys())
tickers = list(sector_etfs.values())
bench_ticker = "SPY"  # benchmark for beta

print("Sector ETFs (Dictionary):")
print("Sectors (keys): ", sector_etfs.keys())
print("Tickers (values): ", sector_etfs.values())

Sector ETFs (Dictionary):
Sectors (keys):  dict_keys(['Communication Services', 'Consumer Discretionary', 'Consumer Staples', 'Energy', 'Financials', 'Health Care', 'Industrials', 'Information Technology', 'Materials', 'Real Estate', 'Utilities'])
Tickers (values):  dict_values(['ESPO', 'ONLN', 'KXI', 'URA', 'ARKF', 'ARKG', 'ARKX', 'QTUM', 'GOAU', 'RWO', 'UTES'])


In [ ]:
# ----------------------------
# 2) Download price data
# ----------------------------

# Download ~5 years of daily adjusted close prices
# Why only 5 years? Longer periods may include structural changes in the market or sectors that could skew beta estimates.

end = dt.date.today()
start = end - dt.timedelta(days=5*365)  # ~5 years
all_tickers = tickers + [bench_ticker]

print(f"Downloading {len(all_tickers)} tickers from {start} to {end} ...")
raw = yf.download(all_tickers, start=start, end=end, progress=False, auto_adjust=True)

# yfinance can return different column layouts:
# - MultiIndex with top-level like 'Adj Close' (common)
# - MultiIndex with top-level like 'Close'
# - Single-level with ticker symbols (when auto_adjust=True it may return adjusted prices directly)
# Handle common cases robustly.
if isinstance(raw.columns, pd.MultiIndex):
    if "Adj Close" in raw.columns.get_level_values(0):
        prices = raw["Adj Close"].copy()
    elif "Close" in raw.columns.get_level_values(0):
        prices = raw["Close"].copy()
    else:
        # fallback: flatten to last level (tickers) and keep requested tickers
        prices = raw.copy()
        prices.columns = raw.columns.get_level_values(-1)
        prices = prices.loc[:, [c for c in prices.columns if c in all_tickers]]
else:
    # single-level columns — assume columns are tickers (or contain them). Keep only requested tickers.
    prices = raw.copy()
    prices = prices.loc[:, [c for c in prices.columns if c in all_tickers]]

# drop columns with no data (if any)
prices = prices.dropna(axis=1, how='all')

# Ensure we have all sector tickers present
missing = [t for t in tickers if t not in prices.columns]
if missing:
    raise RuntimeError(f"Missing data for tickers: {missing}. Check ticker names or network access.")

# Keep only tickers present (in case SPY is missing for some reason)
prices = prices[[t for t in all_tickers if t in prices.columns]]

In [ ]:
# ----------------------------
# 3) Compute returns
# ----------------------------

# Computes daily simple returns and annaulized mean/covariance
# Simple rate of returns: (P_t - P_{t-1}) / P_{t-1}
# Covariance matrix of returns
returns = prices.pct_change().dropna()  # daily simple returns
daily_mu = returns.mean()               # daily mean returns
daily_cov = returns.cov()               # daily covariance
## returns.cov() calcs the pairwise covariance of columns (assets), excluding NA/null values.
## The result is a square matrix where the entry at row i and column j represents the covariance between asset i and asset j.
## Covariance is a measure of how two assets move together. A positive covariance means that the assets tend to move in the same direction, 
## while a negative covariance indicates they move in opposite directions, and zero covariance means no linear relationship,
## and the two variables are uncorrelated.

# Note: The annualized values below are market total returns for the specified time period, which in this case is ~5 years.
trading_days = 252
ann_mu = daily_mu * trading_days        # approximate annualized returns
ann_cov = daily_cov * trading_days      # approximate annualized covariance

# Focus on sector tickers for optimization
mu = ann_mu[tickers]
cov = ann_cov.loc[tickers, tickers]


In [ ]:
# ----------------------------
# 4) Constrained optimization
#    Maximize Sharpe (assume rf=0)
#    Subject to: sum(w)=1, 0 <= w_i <= 0.20
# ----------------------------

# Runs a constrained optimization to maximize Sharpe ratio
# with max 20% allocation per sector ETF for a 100% total allocation portfolio.
# This code section calculates the optimal weights for a diversified portfolio of sector ETFs.

# Sharpe ratio = (prr - rf) / sigma_p
# prr = portfolio rate of return
# rf = risk-free rate of return, typically a short-term government bond yield or a U.S. Treasury bill rate.
# Here we assume rf=0 for simplicity.
# sigma_p = portfolio volatility = sqrt(w^T * Cov * w)

# Sharpe ratio interpretation:
# A higher Sharpe ratio indicates better risk-adjusted performance.
# A Sharpe ratio > 1 is considered good, > 2 very good, > 3 excellent.
# Below 1: indicates suboptimal risk-adjusted returns.

n = len(tickers)
bounds = [(0.0, 0.20) for _ in range(n)]
cons = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0},) # Defines the constraint that the sum of weights equals 1.

# Objective: negative Sharpe (we'll maximize Sharpe)
# *** NOTE: port_ret = expected portfolio return is NOT factoring in risk-free rate here (assumed zero). ***

def neg_sharpe(w, mu, cov):
    port_ret = w.dot(mu) # calcs expected portfolio return
    port_vol = np.sqrt(w @ cov @ w) # calcs portfolio standard deviation (volatility)
    # protect against zero vol
    if port_vol <= 0:
        return 1e6
    return -port_ret / port_vol # negative Sharpe. Why negative? Most optimizers minimize functions, so to maximize Sharpe we minimize its negative.

# Starting guess: equal weights (but feasible? equal = 1/11 ~0.0909 <0.2 is fine)
x0 = np.repeat(1.0 / n, n)

# Runs a constrained optimization to find the portfolio weights that maximize the Sharpe ratio.
## method: Sequential Least Squares Programming (SLSQP), which supports both bounds and constraints.
opt = minimize(neg_sharpe, x0, args=(mu.values, cov.values),
               method='SLSQP', bounds=bounds, constraints=cons,
               options={'ftol':1e-9, 'maxiter':1000})

if not opt.success:
    print("Optimization did not converge. Message:", opt.message)
    # Fallback: use capped proportional allocation (simple heuristic)
    # allocate equal then cap at 0.2 and redistribute remainder proportionally among uncapped
    w = np.repeat(1.0 / n, n)
    caps = np.array([0.20]*n)
    # iterative water-fill redistribution
    def capped_equal(weights, cap):
        w = weights.copy()
        while True:
            over = w > cap
            if not over.any():
                break
            excess = (w[over] - cap[over]).sum()
            w[over] = cap[over]
            free = (~over)
            if free.sum() == 0:
                break
            w[free] = w[free] + excess / free.sum()
        return w
    w = capped_equal(np.repeat(1.0/n, n), caps)
else:
    w = opt.x

weights = pd.Series(w, index=tickers) # creates a pandas Series where earch value in w (the optimized weight) is assigned to its corresponding ticker symbol.
weights.index = [f"{t} ({sector})" for t, sector in zip(tickers, sectors)] # relables index to include ticker and sector name

In [ ]:
# CONTINUE HERE... -ERODRIGUEZ, TUES 25NOV2025 15:41
# ----------------------------
# 5) Portfolio returns and metrics
# ----------------------------
# Build portfolio daily returns from original daily returns (not annualized)
# Ensure alignment (use returns DataFrame)
port_daily_returns = (returns[tickers] @ w)
# annualized return via geometric mean: (prod(1+r))^(trading_days/n) - 1
if len(port_daily_returns) > 0:
    port_ann_return = np.prod(1 + port_daily_returns) ** (trading_days / len(port_daily_returns)) - 1
else:
    port_ann_return = np.nan
port_ann_vol = port_daily_returns.std() * np.sqrt(trading_days)
port_sharpe = (port_ann_return / port_ann_vol) if port_ann_vol>0 else np.nan

In [ ]:

# ----------------------------
# 6) Compute beta vs SPY using OLS (returns regression)
#    Use daily returns aligned
# ----------------------------
# Ensure SPY present in returns
if bench_ticker not in returns.columns:
    raise RuntimeError(f"Benchmark {bench_ticker} not found in downloaded data.")

y = port_daily_returns.dropna()
x = returns[bench_ticker].loc[y.index].dropna()
# align
aligned = pd.concat([y, x], axis=1).dropna()
y_al, x_al = aligned.iloc[:,0], aligned.iloc[:,1]
X = sm.add_constant(x_al.values)
model = sm.OLS(y_al.values, X).fit()
alpha_daily = model.params[0]
beta = model.params[1]
alpha_annual = (1 + alpha_daily) ** trading_days - 1
# approximate annualized beta? beta is scale-invariant; just report beta and daily alpha/annual alpha
# model summary available as model.summary()

# ----------------------------
# 7) Print results and plot
# ----------------------------
pd.set_option('display.float_format', '{:,.4f}'.format)
print("\nOptimized weights (each <= 20%):")
display(weights.sort_values(ascending=False))

print("\nPortfolio annualized return (approx): {:.2%}".format(port_ann_return))
print("Portfolio annualized volatility: {:.2%}".format(port_ann_vol))
print("Portfolio annualized Sharpe (rf=0): {:.3f}".format(port_sharpe))

print("\nRegression vs SPY (daily):")
print(f"  Alpha (daily): {alpha_daily:.6f}")
print(f"  Alpha (annualized): {alpha_annual:.4%}")
print(f"  Beta: {beta:.4f}")
print("\nOLS summary:\n")
print(model.summary())

# Plot weights
plt.figure(figsize=(10,5))
weights.sort_values(ascending=True).plot(kind='barh')
plt.title("Optimized Sector ETF Weights (max 20% each)")
plt.xlabel("Weight")
plt.xlim(0,0.22)
plt.grid(axis='x', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

# Scatter plot portfolio vs SPY daily returns with fitted line
plt.figure(figsize=(7,6))
plt.scatter(x_al, y_al, alpha=0.4, s=10)
xmin, xmax = x_al.min(), x_al.max()
x_line = np.linspace(xmin, xmax, 100)
y_line = alpha_daily + beta * x_line
plt.plot(x_line, y_line, linewidth=2, label=f"Fit: alpha={alpha_daily:.6f}, beta={beta:.3f}")
plt.xlabel("SPY daily return")
plt.ylabel("Portfolio daily return")
plt.title("Portfolio vs SPY (daily returns)")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Optional: show weights as DataFrame
weights_df = pd.DataFrame({
    "Sector": sectors,
    "Ticker": tickers,
    "Weight": w
}).set_index("Ticker")
print("\nWeights table:")
display(weights_df)
